In [1]:
# pip install qiskit==0.31.0

In [2]:
from get_test_datasets import return_dataset
test_dataset = return_dataset()

Generating QPE circuits


In [3]:
len(test_dataset)

400

In [4]:
import tensorflow as tf
from tensorflow import keras

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from helper import *

Using TensorFlow backend.
ibmqfactory.load_account:WARNING:2023-11-27 07:00:20,000: Credentials are already in use. The existing account in the session will be replaced.


{'ibm_lagos': [[0, 1], [1, 0], [1, 2], [1, 3], [2, 1], [3, 1], [3, 5], [4, 5], [5, 3], [5, 4], [5, 6], [6, 5]], 'ibm_nairobi': [[0, 1], [1, 0], [1, 2], [1, 3], [2, 1], [3, 1], [3, 5], [4, 5], [5, 3], [5, 4], [5, 6], [6, 5]], 'ibm_perth': [[0, 1], [1, 0], [1, 2], [1, 3], [2, 1], [3, 1], [3, 5], [4, 5], [5, 3], [5, 4], [5, 6], [6, 5]]}


In [5]:
mlp_edge_feature = tf.keras.models.load_model("models/mlp_edge_features.keras")
dnn = tf.keras.models.load_model("models/dnn.keras")
mlp = tf.keras.models.load_model("models/mlp.keras")

2023-11-27 07:00:23.218948: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2023-11-27 07:00:23.311667: E tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-11-27 07:00:23.311795: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce GTX 1650 Ti computeCapability: 7.5
coreClock: 1.485GHz coreCount: 16 deviceMemorySize: 4.00GiB deviceMemoryBandwidth: 178.84GiB/s
2023-11-27 07:00:23.312056: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory
2023-11-27 07:00:23.312115: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynam

In [6]:
basis_gate_dict = {}
provider = IBMQ.get_provider(hub='ibm-q')
provider.backends(simulator=False)
backends = ['ibm_lagos','ibm_perth','ibm_nairobi']
for backend_name in backends:
    backend = provider.get_backend(backend_name)
    basis_gats = backend.configuration().basis_gates
    basis_gate_dict[backend_name] = basis_gats


In [16]:
def get_transpiled_circ_results(qc, backend, initial_layout):
    basis_gats = basis_gate_dict[backend]
    pass_ = Unroller(basis_gats)
    pm = PassManager(pass_)
    backend = provider.get_backend(backend)
    
    tqc = transpile(qc, backend=backend,initial_layout=initial_layout, optimization_level=0,seed_transpiler=13)
    tqc = pm.run(tqc)
    gates = dict(tqc.count_ops())
    return gates['cx'], tqc.depth()


In [17]:
SS = StandardScaler()
num_qubits = 7
count_or = 0
count_nr = 0
count_mlp = 0
count_mlp_edge = 0
original_depth, original_CNOT = [],[]
mlp_edge_depth, mlp_edge_CNOT = [],[]
dnn_without_ro_depth,dnn_without_ro_CNOT = [],[] 
dnn_with_ro_depth,dnn_with_ro_CNOT = [],[] 
mlp_depth, mlp_CNOT = [],[]
for qc_i in tqdm(range(len(test_dataset[-5:]))):
    qc = test_dataset[qc_i]
    data = datetime.today() - timedelta(days=random.randint(1,150))
    backends = ['ibm_lagos','ibm_perth','ibm_nairobi']
    backend = backends[np.random.randint(0,3)]
    l = add_line(qc,backend, optimization_level=3, refresh=True, show= False, datatime=data)
    
    d={}
    for i in range(len(l[0])):
        d[str(l[0][i])] = l[1][i]
    df = pd.DataFrame(d, index=[0])
    df = clear_dataset(df, 7)
    last_num_qubits = len(df.columns)-num_qubits

    x = SS.fit_transform(df.iloc[:, 2:last_num_qubits].values)
    labels = ((df.iloc[:, last_num_qubits:].values)[0]).tolist()
    
    predicted = dnn.predict(x)
    
    pred_or = np.array(pred_layout(predicted, num_qubits))[0].tolist()
    pred_nr = np.array(pred_layout_diff_elem(predicted,num_qubits))[0].tolist()
    print(pred_or, pred_nr, labels)
    if(pred_or == labels):
        count_or = count_or + 1
    if(pred_nr == labels):
        count_nr = count_nr + 1
    
    mlp_pred = get_labels(np.reshape(mlp.predict(x),(7,7)))
    print(mlp_pred)
    if mlp_pred == labels:
        count_mlp = count_mlp + 1
        
        
    x = SS.fit_transform(get_graph_features(df))
    mlp_edges_pred = get_labels(np.reshape(mlp_edge_feature.predict(x),(7,7)))
    print(mlp_edges_pred)
    if mlp_edges_pred == labels:
        count_mlp_edge = count_mlp_edge + 1
    
    
    # original circuit after unrolling
    cx, depth = get_transpiled_circ_results(qc, backend, initial_layout=labels)
    original_depth.append(depth)
    original_CNOT.append(cx)
    print(original_depth[-1],original_CNOT[-1])
    # NN with edge features
#     qc = test_dataset[qc_i]
    if len(mlp_edges_pred) == len(set(mlp_edges_pred)):
        cx, depth = get_transpiled_circ_results(qc, backend, initial_layout=mlp_edges_pred)
        mlp_edge_depth.append(depth)
        mlp_edge_CNOT.append(cx)
    else:
        mlp_edge_depth.append(float('NaN'))
        mlp_edge_CNOT.append(float('NaN'))
        
    # NN without edge features
    if len(mlp_pred) == len(set(mlp_pred)):
        cx, depth = get_transpiled_circ_results(qc, backend, initial_layout=mlp_pred)
        mlp_depth.append(depth)
        mlp_CNOT.append(cx)
    else:
        mlp_depth.append(float('NaN'))
        mlp_CNOT.append(float('NaN'))
        
    # DNN without repair operator
    if len(pred_or) == len(set(pred_or)):
        cx, depth = get_transpiled_circ_results(qc, backend, initial_layout=pred_or)
        dnn_without_ro_depth.append(depth)
        dnn_without_ro_CNOT.append(cx)
    else:
        dnn_without_ro_depth.append(float("NaN"))
        dnn_without_ro_CNOT.append(float("NaN"))
        
    # DNN with RO
    cx, depth = get_transpiled_circ_results(qc, backend, initial_layout=pred_nr)
    dnn_with_ro_depth.append(depth)
    dnn_with_ro_CNOT.append(cx)
print("DNN accuracy without repair operator:", count_or/len(test_dataset))
print("DNN accuracy with repair operator:", count_nr/len(test_dataset))
print("MLP accuracy:", count_mlp/len(test_dataset))
print("MLP with edge features accuracy:", count_mlp_edge/len(test_dataset))

  0%|                                                                                                                     | 0/5 [00:00<?, ?it/s]

2023-10-22 07:05:04.626054
Backend topology: 2023-10-22 07:05:04.626054
Depth: [106, 106, 86, 86, 102, 102]
not na
[2, 1, 0, 3, 4, 5, 6] [2, 1, 0, 3, 4, 5, 6] [0, 1, 2, 3, 4, 5, 6]
[6, 1, 4, 3, 4, 5, 6]



 20%|█████████████████████▊                                                                                       | 1/5 [00:02<00:09,  2.35s/it]

[2, 1, 0, 3, 4, 5, 6]
60 32
2023-11-18 07:05:06.981760
Backend topology: 2023-11-18 07:05:06.981760
Depth: [190, 190, 86, 86, 86, 86]
not na
[2, 1, 0, 3, 4, 5, 6] [2, 1, 0, 3, 4, 5, 6] [0, 1, 2, 3, 4, 5, 6]
[6, 1, 4, 3, 4, 5, 6]



100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 127.20it/s]

...

2023-08-31 07:05:15.108047
Backend topology: 2023-08-31 07:05:15.108047
Depth: [106, 106, 102, 102, 120, 120]
not na
[2, 1, 0, 3, 4, 5, 6] [2, 1, 0, 3, 4, 5, 6] [0, 1, 2, 3, 4, 5, 6]
[6, 1, 4, 3, 4, 5, 6]



100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 120.22it/s]

[2, 1, 0, 3, 4, 5, 6]
60 32



100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:18<00:00,  3.80s/it]

DNN accuracy without repair operator: 0.0
DNN accuracy with repair operator: 0.0
MLP accuracy: 0.0
MLP with edge features accuracy: 0.0


In [18]:
df = pd.DataFrame()
df["Label Depth"] = original_depth
df["Label CNOTs"] = original_CNOT

df["NN without edge features Depth"] = mlp_depth
df["NN without edge features CNOTs"] = mlp_CNOT

df["NN with edge features Depth"] = mlp_edge_depth
df["NN with edge features CNOTs"] = mlp_edge_CNOT

df["DNN without repair Depth"] = dnn_without_ro_depth
df["DNN without repair CNOTs"] = dnn_without_ro_CNOT

df["DNN with repair Depth"] = dnn_with_ro_depth
df["DNN with repair CNOTs"] = dnn_with_ro_CNOT

In [19]:
df

,Label Depth,Label CNOTs,NN without edge features Depth,NN without edge features CNOTs,NN with edge features Depth,NN with edge features CNOTs,DNN without repair Depth,DNN without repair CNOTs,DNN with repair Depth,DNN with repair CNOTs
0,60,32,NaN,NaN,62,35,62,35,62,35
1,59,44,NaN,NaN,59,44,59,44,59,44
2,57,38,NaN,NaN,57,38,57,38,57,38
3,56,29,NaN,NaN,56,29,56,29,56,29
4,60,32,NaN,NaN,62,35,62,35,62,35


In [20]:
import itertools
layouts = list(itertools.permutations([0,1,2,3,4,5,6]))
qc = test_dataset[-8]
basis_gats = basis_gate_dict['ibm_nairobi']
pass_ = Unroller(basis_gats)
pm = PassManager(pass_)
backend = provider.get_backend('ibm_nairobi')
for l in layouts[:20]:
    tqc = transpile(qc, backend=backend,initial_layout=list(l), optimization_level=0)
    tqc = pm.run(tqc)
    
    print(l,tqc.depth(),dict(tqc.count_ops())['cx'])

(0, 1, 2, 3, 4, 5, 6) 110 78
(0, 1, 2, 3, 4, 6, 5) 108 75
(0, 1, 2, 3, 5, 4, 6) 115 78
(0, 1, 2, 3, 5, 6, 4) 109 72
(0, 1, 2, 3, 6, 4, 5) 103 69
(0, 1, 2, 3, 6, 5, 4) 107 72
(0, 1, 2, 4, 3, 5, 6) 105 75
(0, 1, 2, 4, 3, 6, 5) 104 72
(0, 1, 2, 4, 5, 3, 6) 111 69
(0, 1, 2, 4, 5, 6, 3) 104 72
(0, 1, 2, 4, 6, 3, 5) 105 72
(0, 1, 2, 4, 6, 5, 3) 104 75
(0, 1, 2, 5, 3, 4, 6) 104 69
(0, 1, 2, 5, 3, 6, 4) 104 72
(0, 1, 2, 5, 4, 3, 6) 111 69
(0, 1, 2, 5, 4, 6, 3) 109 78
(0, 1, 2, 5, 6, 3, 4) 106 75
(0, 1, 2, 5, 6, 4, 3) 103 66
(0, 1, 2, 6, 3, 4, 5) 109 78
(0, 1, 2, 6, 3, 5, 4) 105 69
